In [1]:
# Import modules

import pandas as pd
import numpy as np
import sys
import nltk
import pickle

import torch
import torch.nn as nn
import torch.utils.data as utils
import torchwordemb
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import log_loss

from multiprocessing import Pool

sys.path.append('/home/hugoperrin/Bureau/Data science/Kaggle/ToxicComment/Models/')
from CNN_1d import CNN

sys.path.append('/home/hugoperrin/Bureau/Data science/Kaggle/ToxicComment/Models/')
from utils import train, predict

%matplotlib inline

In [2]:
# Import data
data = np.load('/home/hugoperrin/Bureau/Datasets/ToxicComment/Comment2Vec.npy')

Xtrain = pd.read_csv('/home/hugoperrin/Bureau/Datasets/ToxicComment/train.csv')
list_classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
target = Xtrain[list_classes].values

del Xtrain

In [3]:
# Preprocess data for torch
data = data.reshape(data.shape[0],1,data.shape[1])
data.shape
# test_comments = test_comments.reshape(test_comments.shape[0],1,test_comments.shape[1])

(159571, 1, 100)

In [4]:
# Train test split
train_comments = data[:130000,:,:]
valid_comments = data[130001:145000,:,:]
test_comments = data[145001:]

train_labels = target[:130000,:]
valid_labels = target[130001:145000,:]
test_labels = target[145001:,:]

del target, data

In [5]:
%%time

predictions = pd.DataFrame(index=range(len(test_comments)))

for target in list_classes:

    labels_train = train_labels[:,list_classes.index(target)]
    labels_train = labels_train.reshape(labels_train.shape[0],1)

    labels_valid = valid_labels[:,list_classes.index(target)]
    labels_valid = labels_valid.reshape(labels_valid.shape[0],1)

    labels_test = test_labels[:,list_classes.index(target)]
    labels_test = labels_test.reshape(test_labels.shape[0],1)

    use_GPU = True

    batch_size = 512
    num_epoch = 5

    train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(train_comments), 
                                                   torch.FloatTensor(labels_train))

    valid_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(valid_comments), 
                                                   torch.FloatTensor(labels_valid))

    test_dataset = torch.FloatTensor(test_comments)


    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True, 
                                               num_workers = 8)

    valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                               batch_size=batch_size,
                                               shuffle=False, 
                                               num_workers = 8)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=False, 
                                               num_workers = 8)

    net = CNN()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.RMSprop(net.parameters(), lr=0.00001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9)

    train(num_epoch, net, train_loader, optimizer, criterion, valid_loader=valid_loader)

    predictions[target] = predict(net, test_loader)


>> LEARNING: 112691 parameters

Epoch: 1, step:   100, training loss: 0.3259, validation loss: 0.24242
Epoch: 1, step:   200, training loss: 0.2037, validation loss: 0.17444
Epoch: 2, step:   300, training loss: 0.0770, validation loss: 0.16049
Epoch: 2, step:   400, training loss: 0.1563, validation loss: 0.15300
Epoch: 2, step:   500, training loss: 0.1549, validation loss: 0.14829
Epoch: 3, step:   600, training loss: 0.1368, validation loss: 0.14550
Epoch: 3, step:   700, training loss: 0.1483, validation loss: 0.14308
Epoch: 4, step:   800, training loss: 0.0537, validation loss: 0.14172
Epoch: 4, step:   900, training loss: 0.1472, validation loss: 0.14225
Epoch: 4, step:  1000, training loss: 0.1416, validation loss: 0.13838
Epoch: 5, step:  1100, training loss: 0.1184, validation loss: 0.13770
Epoch: 5, step:  1200, training loss: 0.1416, validation loss: 0.13679

>> LEARNING: 112691 parameters

Epoch: 1, step:   100, training loss: 0.1297, validation loss: 0.05245
Epoch: 1, s